# EEG DataLoader

The dimensions of the training set are as follows: 4,500 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_train.

The dimensions of the testing set are as follows: 750 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_test.

You can download it from this Google Drive link: [https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing]. 
Please download the data and place it in your data_path at "./data."

In [132]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [133]:
data_path = './data/'

In [135]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

train_data_mean = np.mean(train_data)
train_data_std = np.std(train_data)
train_data_normalized = (train_data - train_data_mean) / train_data_std
test_data_normalized = (test_data - train_data_mean) / train_data_std

#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data_normalized)
y_train_tensor = torch.LongTensor(train_label)
x_test_tensor = torch.Tensor(test_data_normalized)
y_test_tensor = torch.LongTensor(test_label)

In [136]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

In [137]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

# Build simple Deep learning model

In [210]:
class EEGAutoencoderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EEGAutoencoderClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 795, 384), # Input dimention is 64 channel * 795 time point, and use 256 units for first NN layer
            nn.BatchNorm1d(384),
            nn.LeakyReLU(negative_slope=0.05), # Use LeakyReLu function for NN training 
            nn.Dropout(0.1),
            nn.Linear(384, 128), # 256 NN units to 192 units
            nn.BatchNorm1d(128),
            nn.LeakyReLU(negative_slope=0.05),
            nn.Dropout(0.1),
            nn.Linear(128, 64),#  192 NN units to 128 units
            nn.LeakyReLU(negative_slope=0.05)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, num_classes), # num_classes is 5 (hello,” “help me,” “stop,” “thank you,” and “yes”)
            nn.LogSoftmax(dim=1)  # Use LogSoftmax for multi-class classification
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)
        
        # import pdb;pdb.set_trace()
        x = self.classifier(x)
        return x

In [211]:
num_classes = 5 # setting final output class
model = EEGAutoencoderClassifier(num_classes).to(device) 
criterion = nn.NLLLoss() # Use NLLLoss function to optimize
optimizer = optim.Adam(model.parameters(), lr=0.001) # Setting parameters learning rate = 0.001

In [212]:
num_epochs = 25 # setting training epochs (Number of training iterations)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader: 
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/25, Loss: 1.6079448461532593
Epoch 2/25, Loss: 1.6079834699630737
Epoch 3/25, Loss: 1.5476056337356567
Epoch 4/25, Loss: 1.4990766048431396
Epoch 5/25, Loss: 1.3382047414779663
Epoch 6/25, Loss: 1.3050888776779175
Epoch 7/25, Loss: 1.3496935367584229
Epoch 8/25, Loss: 1.1669254302978516
Epoch 9/25, Loss: 1.0005693435668945
Epoch 10/25, Loss: 0.8613981604576111
Epoch 11/25, Loss: 0.9684632420539856
Epoch 12/25, Loss: 0.8762191534042358
Epoch 13/25, Loss: 0.7137364745140076
Epoch 14/25, Loss: 0.6272343397140503
Epoch 15/25, Loss: 0.7227321863174438
Epoch 16/25, Loss: 0.7768109440803528
Epoch 17/25, Loss: 0.5620742440223694
Epoch 18/25, Loss: 0.6538873314857483
Epoch 19/25, Loss: 0.5923730134963989
Epoch 20/25, Loss: 0.36253905296325684
Epoch 21/25, Loss: 0.3019791543483734
Epoch 22/25, Loss: 0.29663515090942383
Epoch 23/25, Loss: 0.3985271155834198
Epoch 24/25, Loss: 0.34813785552978516
Epoch 25/25, Loss: 0.26108554005622864


In [185]:
model.eval() # Evaluate your model
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 47.02%
